In [7]:
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader

load_dotenv()

True

In [3]:
from services.langchain.chains import load_critisim_chain
from langchain_experimental.plan_and_execute.schema import (
    Plan,
    PlanOutputParser,
    Step,
)
import re

chain = load_critisim_chain(verbose=False)
with open("markdown/sample.md") as f:
    SAMPLE_DOCUMENT = f.read()

output = chain.run(SAMPLE_DOCUMENT)

critisims = output.split("\n\n")

class PlanningOutputParser(PlanOutputParser):
    """Planning output parser."""

    def parse(self, text: str) -> Plan:
        steps = [Step(value=v) for v in re.split("\n\s*\d+\. ", text)[1:]]
        return Plan(steps=steps)

parser = PlanningOutputParser()
plan = parser.parse(output)
plan

Plan(steps=[Step(value="**Lack of Specificity in Cybersecurity Strategy**: The policy outlines the responsibilities of the Cybersecurity Committee, including reviewing and providing oversight on various aspects of the company's cybersecurity posture. However, it does not provide specific details about the cybersecurity strategy, expectations, or measures that will be used to ensure the effectiveness of the cybersecurity program."), Step(value="**Absence of Employee Awareness & Training Program**: The policy does not provide information about the company's approach to cybersecurity awareness and training for employees. Employee behavior is one of the most significant factors in many cybersecurity incidents and breaches, and an effective cybersecurity policy should outline how the company plans to manage this risk."), Step(value='**Lack of Clear Communication Channels**: The policy does not specify the communication channels that will be used to escalate potential security issues or brea

In [5]:
import json

with open("test.json", "r") as f:
    data = json.loads(f.read())
    print(list(data.keys()))

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [8]:
from services.langchain.chains import load_critisim_planner, load_redrafting_chain
from langchain.chat_models import ChatOpenAI
from rich import print

llm = ChatOpenAI(model_name="gpt-4")
planner = load_critisim_planner(llm=llm)
drafter = load_redrafting_chain(llm=llm)

with open("markdown/sample.md") as f:
    SAMPLE_DOCUMENT = f.read()

plan = planner.plan({ "input": SAMPLE_DOCUMENT })

for step in plan.steps:
    print("**Critisim:**", step.value)
    output = drafter({ "critisim": step.value, "context": SAMPLE_DOCUMENT })
    print("Revision:", output["text"])


**Critisim:** **Employee Training**: While the policy mentions employee training, it does not elaborate on what 
this training should include or how often it should be conducted. Cybersecurity is a rapidly changing field, so 
regular, updated training is crucial in keeping up with new threats and technologies.

## NORTHERN TRUST CORPORATION

## CYBERSECURITY RISK OVERSIGHT SUBCOMMITTEE CHARTER

```
Effective April 24, 2023
```

**I. Purpose**.

The purpose of the Cybersecurity Risk Oversight Subcommittee (the “Subcommittee”) is
to assist the Business Risk Committee (the “Committee”) of the Board of Directors (the “Board”)
of Northern Trust Corporation (the “Corporation”) in discharging its oversight duties with
respect to risks related to cybersecurity inherent in the businesses of the Corporation and its
subsidiaries (collectively, “Northern Trust”).

This Subcommittee Charter shall also govern the Subcommittee as and when it acts as the
Cybersecurity Risk Oversight Subcommittee of the Business Risk Committee of the Board of
Directors of The Northern Trust Company.

**II. Subcommittee Membership**.

The Subcommittee shall consist of three or more directors. The Subcommittee’s
membership shall be such that, in the judgment of the Committee, the Subcommittee shall have
the experience, expertise and judgment necessary to discharge its duties and responsibilities
under this Charter. Each Subcommittee member shall be: (a) an “independent director” under
the Corporation’s Corporate Governance Guidelines; and (b) unless otherwise approved by the
Board, a member of the Committee.

Subcommittee members and the Subcommittee Chairman shall be appointed annually by
the Committee (with the approval of the Board, if any proposed member of the Subcommittee is
not also a member of the Committee) taking into consideration the benefits of a regular rotation
in the membership of the Subcommittee, including the Chairmanship, balancing in each case the
need for fresh perspective with the need for experience and continuity. The Committee may fill
vacancies on the Subcommittee and may remove a member from Subcommittee membership at
any time with or without cause.

**III. Subcommittee Structure and Operations**.

```
A. Meetings.
```

The Subcommittee shall meet in person or by telephone conference, videoconference or
other means of communications permitted under applicable Delaware law as frequently as
necessary to fulfill its duties and responsibilities, but no less frequently than quarterly.
Additional meetings may be held, or actions may be taken by unanimous written consent, as
deemed necessary or appropriate by the Subcommittee Chairman or by any other member of the
Subcommittee. Minutes of each meeting shall be prepared by the Secretary or any Assistant
Secretary of the Corporation or such other person designated by the Subcommittee Chairman as
Acting Secretary of the Subcommittee and, when approved, shall be distributed to all Committee

members, as well as all other members of the Board. The minutes shall document the decisions
of the Subcommittee regarding matters related to its duties and responsibilities.

The Subcommittee may meet with any officer of the Corporation, consultants or advisors
as it may deem necessary or appropriate. The Subcommittee may meet in joint or concurrent
sessions with other committees or subcommittees of the Board from time to time to discuss areas
of common interest and significant matters.

```
B. Quorum; Manner of Acting.
```

A majority of the Subcommittee’s members shall constitute a quorum, and the act of a
majority of the members at which a quorum is present shall be the act of the Subcommittee. In the
event of a tie vote on any issue, the Subcommittee Chairman’s vote shall decide the issue. In the
absence or disqualification of a member of the Subcommittee, the members present at any meeting
and not disqualified from voting, whether or not they constitute a quorum, may unanimously
appoint another member of the Committee (or of the Board, with the approval of the Board) who
meets the Subcommittee membership requirements set forth herein to act at the meeting in the
place of any such absent or disqualified member.

```
C. Resources.
```

The Subcommittee shall have the resources and authority appropriate to disch

**Critisim:** **Vendor Management**: The policy does not include any mention of how vendors and third-party 
suppliers are to be managed from a cybersecurity perspective. These entities can often pose a significant risk and 
should be included in the overall cybersecurity strategy.